# Reading and Cleaning data

## Reading Data

- Reading data is the first step to any data analysis task.
- Pandas offers versatile functions to read data from various file formats.
  
  **Example for reading from an Excel file**:
  ```python
  df = pd.read_excel('input.xlsx', sheet_name='Sheet1')
  df
  ```

  **Example for reading from a CSV file**:
  ```python
  df = pd.read_csv('data/raw/input.csv')
  df
  ```

- **File Formats**:
  - Pandas provides functions to read from a wide range of sources, including:
    - Text formats such as CSV, JSON, and HTML. Use `pd.read_csv()`, `pd.read_json()`, and `pd.read_html()` respectively.
    - Binary formats such as Excel, HDF5, and Parquet. Use `pd.read_excel()`, `pd.read_hdf()`, and `pd.read_parquet()` respectively.
    - SQL databases like SQLite, PostgreSQL, and MySQL. For this, a connection needs to be established with the respective database and then use `pd.read_sql_query()` or `pd.read_sql_table()` to fetch the data.

Remember, when reading data, you need to specify the correct filepath to the data that you want to read. The filepath is the location of the file on your computer. If you are using Google Colab, you need to:
1. mount your drive
2. set your working directory
3. specify the filepath to the data on your Google Drive.

Lets start by importing the maize data and having a look at it - it comes from the South African Abstract of Agricultural Statistics.

In [3]:
import pandas as pd
import numpy as np
import os

#set the working directory
os.chdir("/Users/jancg/Library/CloudStorage/OneDrive-StellenboschUniversity/3_LE/3_Courses/AE_Python")

#read the data
df_mz = pd.read_csv("data/raw/SA_maize.csv")

Now that we've imported the data, we need to have a look at the data so check the data types and the first few rows of the data.

In [4]:
print(df_mz.head())

  prod year Area planted production      Value price white price yellow
0   1980/81        4 488     14 872  1 768 711      134.15          134
1   1981/82        4 664      8 781  1 190 204      155.05       155.05
2   1982/83        4 680      4 399    770 447      170.05       170.05
3   1983/84        4 839      4 797  1 055 662       219.5       215.55
4   1984/85        4 502      8 444  1 920 603      221.45        217.5


Ok, so we've got a production year variable and then a some indicators for each year. A couple of observations:

1. Some of the variable names have spaces in them, which is not ideal, we will need deal with the spaces
2. ome of the column names have capital letters in them, which is also not ideal, as a rule of thumb, we should always use lower case letters for column names.
3. it seems that some or all of the year values are separated by `/`, this can cause problems when we want to do calculations with the data. We need to address it.
4. we need to check the data types since we want to do some calculations with the data and we need to make sure that the data types are correct. The fact that there spaces as 1000 separators in the data means that the data types are probably not numeric.
5. here is a column called `value`, this is not a very descriptive name, we should change it to something more descriptive.

**1. & 2.** Lets start by renaming the columns to remove the spaces. Lets just look for the spaces in the column names and replace them with `_`. Lets also deal with the capital letters at the same time.

In [5]:
#replace spaces in column names with underscores
df_mz.columns = df_mz.columns.str.replace(' ', '_')
#all lower case
df_mz.columns = df_mz.columns.str.lower()

print(df_mz.columns)

Index(['prod_year', 'area_planted', 'production', 'value', 'price_white',
       'price_yellow'],
      dtype='object')


**3.** Now lets replace the `/` with a `_` in the production year column

In [6]:
df_mz['prod_year'] = df_mz['prod_year'].str.replace('/', '_')
print(df_mz['prod_year'].head())

0    1980_81
1    1981_82
2    1982_83
3    1983_84
4    1984_85
Name: prod_year, dtype: object


**4.** Now lets check the data types and takes the necessary steps if they are incorrect.

In [7]:
print(df_mz.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prod_year     42 non-null     object
 1   area_planted  42 non-null     object
 2   production    42 non-null     object
 3   value         42 non-null     object
 4   price_white   42 non-null     object
 5   price_yellow  42 non-null     object
dtypes: object(6)
memory usage: 2.1+ KB
None


Ok, so they are objects which is not what we want, and we cannot convert them to numeric because of the spaces. Lets remove the spaces and convert the data types to numeric.

In [8]:
#remove spaces
df_mz['area_planted'] = df_mz['area_planted'].str.replace(' ', '')
df_mz['production'] = df_mz['production'].str.replace(' ', '')
df_mz['value'] = df_mz['value'].str.replace(' ', '')
df_mz['price_white'] = df_mz['price_white'].str.replace(' ', '')
df_mz['price_yellow'] = df_mz['price_yellow'].str.replace(' ', '')

#convert to numeric
df_mz['area_planted'] = pd.to_numeric(df_mz['area_planted'])
df_mz['production'] = pd.to_numeric(df_mz['production'])
df_mz['value'] = pd.to_numeric(df_mz['value'])
df_mz['price_white'] = pd.to_numeric(df_mz['price_white'])
df_mz['price_yellow'] = pd.to_numeric(df_mz['price_yellow'])

**5.** Lastly, lets rename the `value` column to something more descriptive.

In [9]:
#rename value column
df_mz = df_mz.rename(columns={'value': 'output_value_rands'})

### Calculating Yield

Lets calculate the maize yield for each year and add it to the data frame.

In [10]:
df_mz['yield'] = df_mz['production'] / df_mz['area_planted']

Using the describe function, we can see summary statistics for the yield variable.

In [11]:
df_mz['yield'].describe()

count    42.000000
mean      3.082552
std       1.315369
min       0.785286
25%       2.210766
50%       2.850034
75%       3.958561
max       5.860100
Name: yield, dtype: float64

So the we have 42 years of data with an average yield of 3.08 tons per hectare. The minimum yield was 0.78 tons per hectare and the maximum yield was 5.8 tons per hectare.

## Wide and Long Data Types in Data Manipulation

When working with data, especially in a tabular format like dataframes in Python, it's common to encounter two primary structures: **wide format** and **long format** (sometimes called "tidy" format). These structures refer to how data is organized in rows and columns. Understanding the distinction between the two is crucial for effective data manipulation and visualization in Python.

### Wide Format

In a **wide format**, a single subject's repeated observations are spread across multiple columns. It’s called ‘wide’ because you might have a wide table with many columns.

**Example:**

Suppose we have a dataset tracking the sales of a product for three months: January, February, and March.

```
| Subject | January | February | March |
|---------|---------|----------|-------|
| Product A | 10      | 12       | 11    |
| Product B | 15      | 17       | 18    |
```

Here, the sales numbers for each month are spread horizontally in separate columns.

### Long (Tidy) Format

In a **long format**, each row is a single observation, with one column specifying the "type" of observation and another column specifying the value. This format is often preferred in data visualization tools and can make data manipulation more straightforward in some contexts.

Using the same sales data:

```
| Subject   | Month   | Sales |
|-----------|---------|-------|
| Product A | January | 10    |
| Product A | February| 12    |
| Product A | March   | 11    |
| Product B | January | 15    |
| Product B | February| 17    |
| Product B | March   | 18    |
```

Each row corresponds to a single observation - a month's sales for a product.

### Conversion with Pandas

The Python library `pandas` offers intuitive functions to convert between wide and long formats:

- **melt**: This function is used to change data from wide to long format.
- **pivot** or **pivot_table**: These functions can convert data from long to wide format.

#### Example:

```python
import pandas as pd

# Wide to Long format using melt
df_wide = pd.DataFrame({
    'Subject': ['Product A', 'Product B'],
    'January': [10, 15],
    'February': [12, 17],
    'March': [11, 18]
})

df_long = df_wide.melt(id_vars=['Subject'], 
                       value_vars=['January', 'February', 'March'], 
                       var_name='Month', 
                       value_name='Sales')

# Long to Wide format using pivot
df_wide_again = df_long.pivot(index='Subject', columns='Month', values='Sales').reset_index()
```

### When to Use Which Format?

- **Wide Format**: Useful when comparing multiple variables for a single subject. It's often preferred in tools like Excel and in specific scenarios in data analysis where having data in separate columns is beneficial.
  
- **Long Format**: It's a preferred format for many data visualization tools like `seaborn` and `ggplot2` in R. It's also conducive for various types of data transformations and is considered a "tidier" way to represent data where each variable forms a column, each observation forms a row, and each type of observational unit forms a table.

Understanding these data formats and how to convert between them can greatly improve your efficiency and effectiveness in data manipulation and analysis in Python.

Lets make the maize data long:

In [12]:
df_mz_long = df_mz.melt(id_vars=['prod_year'], 
                       var_name='variable', 
                       value_name='value')

#Lets add the crop
df_mz_long['crop'] = 'maize'

----

## Tutorial 3, Part 1: Reading and Cleaning Data

Now it your turn, do the following:
1. Import the SA_soybean data and call it df_soy
2. Correct the column names
3. Correct the production year column
4. Check the data types and convert them to numeric if necessary
5. Rename the value column to something more descriptive
6. Calculate the soybean yield for each year and add it to the data frame
7. Make the data long, calling it df_soy_long
8. Add a `crop_type` column to the long data frame and set it to `soybean`

----
----

## Merging and Joining Data

We often want to combine different data sources to create a single dataset for analysis. For instance, we may have data on sales for different products in different regions, and we want to combine them to get a single dataset containing sales for all products across all regions. This process is called **data merging** or **data joining**.

### Basics of Joining Data

Pandas provides various ways to combine DataFrames including `.merge()` for database-style joins. The `merge()` function takes the following arguments:

```python
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False)
```

- `left`: A DataFrame object.
- `right`: Another DataFrame object.
- `how`: One of 'left', 'right', 'outer', 'inner'. Defaults to 'inner', which keeps only the rows where the merge "on" value exists in both the left and right DataFrames.
- `on`: Columns (names) to join on. Must be found in both the left and right DataFrame objects.
- `left_on`: Columns from the left DataFrame to use as keys.
- `right_on`: Columns from the right DataFrame to use as keys.
- `left_index`: If `True`, use the index (row labels) from the left DataFrame as its join key(s). In other words, use the index to match rows from the left DataFrame with rows from the right DataFrame.
- `right_index`: Same usage as `left_index` for the right DataFrame.

Lets look at a couple of examples, but first, lets construct an example dataframe:

In [ ]:
# Data
data = {
    'Full Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age in Years': [25, 30, 35, 40, 45],
    'Occupation': ['Mechanical Engineer', 'Physician', 'Artist', 'Lawyer', 'Scientist'],
    'Age_squared': [625, 900, 1225, 1600, 2025],
    'Name_length': [5, 3, 7, 5, 3],
    'Is_Elderly': ['No', 'No', 'No', 'No', 'Yes'],
    'Life_Stage': ['Young', 'Middle-aged', 'Middle-aged', 'Middle-aged', 'Middle-aged']
}

# Creating the DataFrame
df = pd.DataFrame(data)
print(df)
print("")
print("The Merge data is:")
print("")
data1 = {'Full Name': ['Alice', 'Bob','Peter'], 'Salary': [50000, 60000,100000]}
df_add = pd.DataFrame(data1)
print(df_add)


  Full Name  Age in Years           Occupation  Age_squared  Name_length  \
0     Alice            25  Mechanical Engineer          625            5   
1       Bob            30            Physician          900            3   
2   Charlie            35               Artist         1225            7   
3     David            40               Lawyer         1600            5   
4       Eva            45            Scientist         2025            3   

  Is_Elderly   Life_Stage  
0         No        Young  
1         No  Middle-aged  
2         No  Middle-aged  
3         No  Middle-aged  
4        Yes  Middle-aged  

The Merge data is:

  Full Name  Salary
0     Alice   50000
1       Bob   60000
2     Peter  100000


In [ ]:
# Example 1: Inner join on `Full Name`
merged_df = pd.merge(df, df_add, on='Full Name',how = 'inner')
print(merged_df)

  Full Name  Age in Years           Occupation  Age_squared  Name_length  \
0     Alice            25  Mechanical Engineer          625            5   
1       Bob            30            Physician          900            3   

  Is_Elderly   Life_Stage  Salary  
0         No        Young   50000  
1         No  Middle-aged   60000  


In [ ]:
# Example 2: Left join
left_joined_df = pd.merge(df, df_add, on='Full Name', how='left')
print(left_joined_df)

  Full Name  Age in Years           Occupation  Age_squared  Name_length  \
0     Alice            25  Mechanical Engineer          625            5   
1       Bob            30            Physician          900            3   
2   Charlie            35               Artist         1225            7   
3     David            40               Lawyer         1600            5   
4       Eva            45            Scientist         2025            3   

  Is_Elderly   Life_Stage   Salary  
0         No        Young  50000.0  
1         No  Middle-aged  60000.0  
2         No  Middle-aged      NaN  
3         No  Middle-aged      NaN  
4        Yes  Middle-aged      NaN  


In [ ]:
# Example 3: Outer join
outer_joined_df = pd.merge(df, df_add, on='Full Name', how='outer')
print(outer_joined_df)

  Full Name  Age in Years           Occupation  Age_squared  Name_length  \
0     Alice          25.0  Mechanical Engineer        625.0          5.0   
1       Bob          30.0            Physician        900.0          3.0   
2   Charlie          35.0               Artist       1225.0          7.0   
3     David          40.0               Lawyer       1600.0          5.0   
4       Eva          45.0            Scientist       2025.0          3.0   
5     Peter           NaN                  NaN          NaN          NaN   

  Is_Elderly   Life_Stage    Salary  
0         No        Young   50000.0  
1         No  Middle-aged   60000.0  
2         No  Middle-aged       NaN  
3         No  Middle-aged       NaN  
4        Yes  Middle-aged       NaN  
5        NaN          NaN  100000.0  


----
## Tutorial 3, Part 2: Merging data

Now it your turn, do the following:

1. Import the `SA_season_type.csv` data and call it `df_season`
2. Merge it with the df_maize_long data and call it `df_maize_long_season`
3. Merge it with the df_soy_long data and call it `df_soy_long_season`

**Note:** Make sure that you use the correct join type so that you do not lose data or introduce NA values.

----
----

## Concatenating Data

Concatenating data is the process of combining data from multiple sources into a single table. This is a common task in data science and data analysis scenarios, and `pandas` provides several functions to make this process straightforward.

### Concatenating DataFrames

The `pandas` function `concat()` can be used to concatenate two or more DataFrames into a single DataFrame. The function takes a list of DataFrames as an argument and returns a single DataFrame that combines the rows of the input DataFrames.

In [13]:
df1 = pd.DataFrame({
    'Subject': ['Product A', 'Product B'],
    'January': [10, 15],
    'February': [12, 17],
    'March': [11, 18]
})

df2 = pd.DataFrame({
    'Subject': ['Product C', 'Product D'],
    'January': [20, 25],
    'February': [22, 27],
    'March': [21, 28]
})

df3 = pd.concat([df1, df2], ignore_index=True)
print(df3)

     Subject  January  February  March
0  Product A       10        12     11
1  Product B       15        17     18
2  Product C       20        22     21
3  Product D       25        27     28


Lets concatenate the maize and sunflower data.

In [16]:
df_sunf = pd.read_csv("data/raw/SA_sunflower.csv")

df_sunf_long = df_sunf.melt(id_vars=['prod_year'],var_name='variable', value_name='value')

df_sunf_long['crop'] = 'sunflower'

df_mz_sunf = pd.concat([df_mz_long, df_sunf_long], ignore_index=True)
print(df_mz_sunf.head())

print(df_mz_sunf['prod_year'].unique())

  prod_year      variable   value   crop
0   1980_81  area_planted  4488.0  maize
1   1981_82  area_planted  4664.0  maize
2   1982_83  area_planted  4680.0  maize
3   1983_84  area_planted  4839.0  maize
4   1984_85  area_planted  4502.0  maize
['1980_81' '1981_82' '1982_83' '1983_84' '1984_85' '1985_86' '1986_87'
 '1987_88' '1988_89' '1989_90' '1990_91' '1991_92' '1992_93' '1993_94'
 '1994_95' '1995_96' '1996_97' '1997_98' '1998_99' '1999_00' '2000_01'
 '2001_02' '2002_03' '2003_04' '2004_05' '2005_06' '2006_07' '2007_08'
 '2008_09' '2009_10' '2010_11' '2011_12' '2012_13' '2013_14' '2014_15'
 '2015_16' '2016_17' '2017_18' '2018_19' '2019_20' '2020_21' '2021_22']


## Writing Data
  - DataFrames can be saved to a variety of file formats.
  - Example for writing to an Excel file:
    ```python
    df.to_excel('output.xlsx', sheet_name='Sheet1', index=False)
    ```

- **File Formats**:
  - Pandas supports a variety of file formats including:
    - Text formats such as CSV, JSON, and HTML.
    - Binary formats such as Excel, HDF5, and Parquet.
    - SQL databases like SQLite, PostgreSQL, and MySQL.

----
## Tutorial 3, Part 3: Writing data

Ok, please wrap up your tutorial by concatenating the maize, soybean and sunflower data and writing it to a csv file. Save it somewhere on your Google Drive where you can find it again since we will be using it in the next tutorial.

----
----